In [1]:
import sys
sys.path.insert(1,'../tools')
import pandas as pd
import numpy as np
import html
import random
from IPython.core.display import display, HTML
import utils
import matplotlib.pyplot as plt

In [2]:
from torch.utils.data import Dataset, DataLoader
from transformers import modeling_bert
import numpy as np
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tqdm import tqdm
import pdb
import torch.nn.functional as F
from torch import optim
import time
import imp
import os
from sklearn.metrics import confusion_matrix
import json
import sklearn

In [3]:
imp.reload(utils)
hpara1 = utils.hpara()
hpara1.ratio = 2
hpara1.batch_size = 4
hpara1.word_layers=4
hpara1.sent_layers=1
hpara1.use_position_embedding = False
hpara1.word_lr = 1e-04 #5e-05
hpara1.sent_lr = 8e-04 #2e-04
hpara1.decay_step = 10
hpara1.decay_gamma = 0.5
hpara1.max_sent_len = 100
hpara1.fix_cls = True
hpara1.head_num = 12
hpara1.att_decay_step=3
hpara1.att_decay_rate=0.5
hpara1.max_doc_len = 300

In [4]:
hpara1.use_SSI_Bert = False
hpara1.use_narrow = False
hpara1.use_PU_Bert = False


In [5]:
hpara1.use_PU_Bert

False

In [ ]:
with open('./exp/best_clinical_bert/hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)


In [6]:
pretrain_model_dir = '../BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000'
vocab_file = os.path.join(pretrain_model_dir,'vocab.txt')
tokenizer = utils._load_tf_tokenizer(vocab_file = vocab_file)
bert_config_file = os.path.join(pretrain_model_dir,'bert_config.json')

trained_word_model = './exp/Clinical_Bert_2020_12_31_100/save_word_4.bin' 
trained_sent_model = './exp/Clinical_Bert_2020_12_31_100/save_sent_4.bin' 

if hpara1.use_SSI_Bert and 'SSI' not in trained_word_model:
    print('ERROR')
if not hpara1.use_SSI_Bert and 'SSI'  in trained_word_model:
    print('ERROR')

In [7]:
config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)
model_word.load_state_dict(torch.load(trained_word_model))
cls_weight = model_word.state_dict()['embeddings.word_embeddings.weight'][101]

In [8]:
config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
config_doc.num_attention_heads = hpara1.head_num
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,cls_weight,use_position_embedding=use_position_embedding)
model_sent.load_state_dict(torch.load(trained_sent_model))

<All keys matched successfully>

In [9]:
model_word = model_word.cuda()
model_sent = model_sent.cuda()

In [ ]:
_,_,_,label_test = utils.load_raw_data_new()
svm_y_pred = np.load('../PU_classifier_training/svm_y_pred.npy')
bert_y_pred = np.load('y_hat_clinical_noN.npy')[-len(label_test):]
svm_thres = 0.37
bert_thres = 0.64
diff_svm = (svm_y_pred > svm_thres) - np.array(label_test)
diff_bert = (bert_y_pred > bert_thres) - np.array(label_test)
fn_review = set(np.where(diff_svm == -1)[0]).intersection(np.where(diff_bert == -1)[0])
print(fn_review)

In [ ]:
# False positive examples
fp_review = {38, 44, 60, 71, 137, 270, 311, 329, 393, 449, 461, 513, 523, 592, 625, 662, 666, 705}

In [10]:
def write_key_sent(sent_weight,doc,write_file,index):
    log = 'Doc {}: <br />key sentence1: {}, score: {}, <br />key sentence2: {}, score: {}, <br />key sentence3: {}, score: {} <br />'
    key_sent_index = np.where(sent_weight[1:-1]>0.001)[0] + 1
    key_sent_index = key_sent_index[np.argsort(sent_weight[key_sent_index])].tolist()
    doc.append('SEP')
    doc.insert(0,'CLS')        
    #pdb.set_trace()
    if len(key_sent_index) == 0:
        key_sent_index = [np.argmax(sent_weight[:])]
    if len(key_sent_index) <3:
        [key_sent_index.append(key_sent_index[0]) for i in range(3 - len(key_sent_index))]

    sent1 = doc[key_sent_index[-1]]
    weight1 = sent_weight[key_sent_index[-1]]

    sent2 = doc[key_sent_index[-2]]
    weight2 = sent_weight[key_sent_index[-2]]

    sent3 = doc[key_sent_index[-3]]
    weight3 = sent_weight[key_sent_index[-3]]

    to_print = log.format(index,sent1,weight1,sent2,weight2,sent3,weight3)
    write_file.writelines(to_print)
    write_file.close()

In [ ]:
import pickle
src_dir = '../preprocessing/result/ratio2/'
text_test  = pickle.load(open(src_dir+'merged_text_test.pkl','rb')) 
label_test = pickle.load(open(src_dir+'label_test.pkl','rb'))
##Change here!
indices_to_review = fn_review
text_to_review = []
label_to_review = []
for ind in indices_to_review:
    text_to_review.append(text_test[ind])
    label_to_review.append(label_test[ind])
params_all = {'batch_size': 1,
    'shuffle': False,
    'num_workers': 6}
review_set = utils.DocDataset(text_to_review, label_to_review)
review_generator = DataLoader(review_set, **params_all)

In [11]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './statis/'+date[1:-1]+'/'
    
if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'used_model.txt','w') as f:
    f.writelines(trained_word_model + '\n' +trained_sent_model)
    
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
max_epoch = hpara1.max_epoch
log = 'Iter {}/{}, Loss={:.3f},accu={:.3f},time={:.3f}\n'
from tqdm import tqdm
batch_size = hpara1.batch_size
accumulation_steps = hpara1.accumulation_steps
max_sent_len = hpara1.max_sent_len
max_doc_len = hpara1.max_doc_len
#progress_bar = tqdm(enumerate(training_generator))
para_dict = {}
hpara_list = []

In [12]:
hpara1.max_doc_len

300

In [13]:
src_path = '../preprocessing/result/pu_chart/day3_data.csv'
training_generator,validation_generator = utils.load_df_data(src_path)

In [14]:
model_sent.eval()
model_word.eval()
pred_list = []
y_list = []
y_hat = []
total_num = 0
correct = 0
max_doc_len = hpara1.max_doc_len
batch_size = hpara1.batch_size
narrow = hpara1.use_narrow
def run_model(doc,label):
    label = label.cuda()
    batch_count=0
    sent_num = 0
    end_ind = 0
    input_tensors = torch.zeros([1,max_doc_len,hpara1.hidden_size]).cuda()
    while end_ind <= len(doc) and end_ind < max_doc_len-1:
        batch_sent = utils.batch_sent_loader(doc,batch_size,batch_count,max_doc_len=max_doc_len)
        cur_batch_size = len(batch_sent)
        sent_num += cur_batch_size
        input_ids = torch.zeros(cur_batch_size,max_sent_len).long().cuda()
        input_mask = torch.ones(cur_batch_size,max_sent_len).long().cuda()
        for i in range(len(batch_sent)):
            tmp_ids,tmp_mask= utils.word_tokenize(batch_sent[i][0],max_sent_len,tokenizer)
            input_ids[i,:] = torch.tensor(tmp_ids)
            input_mask[i,:] = torch.tensor(tmp_mask)
        _,tmp_input_tensors,word_att_output = model_word(input_ids,attention_mask=input_mask)
        start_ind = batch_count*batch_size+1
        end_ind = start_ind + cur_batch_size
        input_tensors[0,start_ind:end_ind] = tmp_input_tensors
        batch_count += 1

    sent_mask = [1]*(end_ind)
    while len(sent_mask)<max_doc_len:
        sent_mask.append(0)
    sent_mask = torch.tensor(sent_mask).unsqueeze(0).cuda()
    if narrow:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask,epoch=cur_epoch)
    else:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask)
    return sent_att_output,doc,proba,label

In [15]:
label.item()


1

In [19]:
np.exp(proba)

array([[0.01564184, 0.9843582 ]], dtype=float32)

In [21]:
import pdb
max_alpha = 0.6 
colored_text = '<span style="background-color:rgba({},100,{},{} ); line-height:30px">{}</span>' 
highlighted_text = []
y_list = []
y_hat = []
proba = []
pred_list= []
fp_proba_list = []
thres = 0.5
count = 0

for doc_count,(doc,label) in enumerate(validation_generator):
    sent_att_output,doc,proba,label = run_model(doc,label)
    proba = proba.cpu().detach().numpy()
    proba = np.exp(proba)
    pred_list.append(proba[0,1]>thres)
    is_wrong = (proba[0,1]>thres) - label.cpu().numpy()
    sent_weight = utils.One_layer_back_trace(sent_att_output[-1])
    sent_weight = sent_weight[0]

    max_alpha = 0.1
    highlighted_text = []
    for i in range(min(299,len(doc))):
        sent = doc[i][0]
        weight = sent_weight[i+1]

        red_value =  weight/ max(sent_weight)*255
        blue_value = 255 - red_value
        color_intensity = max(max_alpha, weight/ max(sent_weight))
        new_colored_text = colored_text.format(red_value,blue_value,color_intensity,html.escape(sent+'\n'))
        highlighted_text.append(new_colored_text)
        #highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(sent+'\n') + '</span>')

    highlighted_text = '<br />'.join(highlighted_text)
    is_wrong = 1
    #display(HTML(highlighted_text))
    if (is_wrong == 1):
        count +=1
        write_file =  open(save_dir + str(count) + '.html','w')
        write_key_sent(sent_weight,doc,write_file,count)

        fp_proba_list.append(proba[0,1])
        print(str(count) +': ' +str(label.item()) + ' pred: ' + str(proba[0,1]))
        with open(save_dir + str(count) + '.html','a') as f:
            f.writelines(highlighted_text)

        #print(proba)
        #pdb.set_trace()
        

1: 1 pred: 0.9584702
2: 1 pred: 0.9709317
3: 1 pred: 0.9699949
4: 1 pred: 0.9027958
5: 1 pred: 0.97901434
6: 1 pred: 0.9569874
7: 1 pred: 0.98345643
8: 1 pred: 0.9859784
9: 1 pred: 0.93151975
10: 1 pred: 0.40532994
11: 1 pred: 0.8582049
12: 1 pred: 0.9613161
13: 1 pred: 0.97552705
14: 1 pred: 0.9547977
15: 1 pred: 0.96432364
16: 1 pred: 0.89845717
17: 1 pred: 0.9843582
18: 1 pred: 0.9825404
19: 1 pred: 0.6478961
20: 1 pred: 0.45358464
21: 1 pred: 0.89051807
22: 1 pred: 0.98680496
23: 1 pred: 0.2325388
24: 1 pred: 0.9565162
25: 1 pred: 0.9747755
26: 1 pred: 0.41038743
27: 1 pred: 0.9194095
28: 1 pred: 0.97129977
29: 1 pred: 0.99327224
30: 1 pred: 0.97740096
31: 1 pred: 0.9960855
32: 1 pred: 0.8299135
33: 1 pred: 0.9472728
34: 1 pred: 0.98844224
35: 1 pred: 0.68086404
36: 1 pred: 0.72467536
37: 1 pred: 0.9334986
38: 1 pred: 0.99166745
39: 1 pred: 0.983453
40: 1 pred: 0.98985696
41: 1 pred: 0.9660672
42: 1 pred: 0.98975116
43: 1 pred: 0.99186265
44: 1 pred: 0.963542
45: 1 pred: 0.9843083


357: 1 pred: 0.80733
358: 1 pred: 0.038526095
359: 1 pred: 0.6226448
360: 1 pred: 0.16778679
361: 1 pred: 0.8810266
362: 1 pred: 0.36789173
363: 1 pred: 0.9010521
364: 1 pred: 0.25404793
365: 1 pred: 0.6690386
366: 1 pred: 0.04293535
367: 1 pred: 0.76841104
368: 1 pred: 0.7731232
369: 1 pred: 0.7326093
370: 1 pred: 0.8462063
371: 1 pred: 0.5128243
372: 1 pred: 0.9533096
373: 1 pred: 0.7202774
374: 1 pred: 0.17532352
375: 1 pred: 0.81797194
376: 1 pred: 0.8163145
377: 1 pred: 0.90389264
378: 1 pred: 0.8083337
379: 1 pred: 0.79480696
380: 1 pred: 0.80797344
381: 1 pred: 0.84262955
382: 1 pred: 0.97379893
383: 1 pred: 0.68348277
384: 1 pred: 0.14997622
385: 1 pred: 0.7699531
386: 1 pred: 0.87496537
387: 1 pred: 0.9072356
388: 1 pred: 0.8351886
389: 1 pred: 0.7848909
390: 1 pred: 0.89508754
391: 1 pred: 0.9086953
392: 1 pred: 0.89898336
393: 1 pred: 0.09732714
394: 1 pred: 0.87815607
395: 1 pred: 0.70865273
396: 1 pred: 0.7226535
397: 1 pred: 0.02551825
398: 1 pred: 0.73193187
399: 1 pred:

695: 0 pred: 0.006627143
696: 0 pred: 0.006746917
697: 0 pred: 0.07344129
698: 0 pred: 0.0070553874
699: 0 pred: 0.07370441
700: 0 pred: 0.47748655
701: 0 pred: 0.008965437
702: 0 pred: 0.007282106
703: 0 pred: 0.0057267905
704: 0 pred: 0.6636743
705: 0 pred: 0.007395813
706: 0 pred: 0.007336995
707: 0 pred: 0.0069187856
708: 0 pred: 0.007708367
709: 0 pred: 0.008889855
710: 0 pred: 0.12047636
711: 0 pred: 0.014093332
712: 0 pred: 0.045688532
713: 0 pred: 0.008558585
714: 0 pred: 0.009642321
715: 0 pred: 0.0356312
716: 0 pred: 0.006976123
717: 0 pred: 0.005520805
718: 0 pred: 0.0063074557
719: 0 pred: 0.0066430694
720: 0 pred: 0.0072922944
721: 0 pred: 0.17472093
722: 0 pred: 0.006108413
723: 0 pred: 0.004322116
724: 0 pred: 0.0118632885
725: 0 pred: 0.71641576
726: 0 pred: 0.022672648
727: 0 pred: 0.010747476
728: 0 pred: 0.0070496537
729: 0 pred: 0.008595012
730: 0 pred: 0.007830062
731: 0 pred: 0.0067251367
732: 0 pred: 0.017048575
733: 0 pred: 0.017197551
734: 0 pred: 0.019881295
7

1028: 0 pred: 0.007550937
1029: 0 pred: 0.0107069025
1030: 0 pred: 0.00783905
1031: 0 pred: 0.63658947
1032: 0 pred: 0.004989763
1033: 0 pred: 0.075854175
1034: 0 pred: 0.006596388
1035: 0 pred: 0.008476406
1036: 0 pred: 0.01602337
1037: 0 pred: 0.006249517
1038: 0 pred: 0.009182867
1039: 0 pred: 0.029082293
1040: 0 pred: 0.006166287
1041: 0 pred: 0.2505009
1042: 0 pred: 0.07104752
1043: 0 pred: 0.5404702
1044: 0 pred: 0.6535209
1045: 0 pred: 0.6096613
1046: 0 pred: 0.98182756
1047: 0 pred: 0.077800326
1048: 0 pred: 0.0076750787
1049: 0 pred: 0.01460594
1050: 0 pred: 0.0068727317
1051: 0 pred: 0.056090634
1052: 0 pred: 0.009072807
1053: 0 pred: 0.015865413
1054: 0 pred: 0.16487095
1055: 0 pred: 0.57348746
1056: 0 pred: 0.014695578
1057: 0 pred: 0.0077475416
1058: 0 pred: 0.01138786
1059: 0 pred: 0.02147974
1060: 0 pred: 0.00614758
1061: 0 pred: 0.006121908
1062: 0 pred: 0.016677987
1063: 0 pred: 0.01422686
1064: 0 pred: 0.006171029
1065: 0 pred: 0.005836061
1066: 0 pred: 0.010309098
10

1348: 0 pred: 0.7408151
1349: 0 pred: 0.018017543
1350: 0 pred: 0.011142483
1351: 0 pred: 0.0066546192
1352: 0 pred: 0.055041637
1353: 0 pred: 0.5083355
1354: 0 pred: 0.007433293
1355: 0 pred: 0.025212819
1356: 0 pred: 0.02525566
1357: 0 pred: 0.00632814
1358: 0 pred: 0.013549316
1359: 0 pred: 0.02586655
1360: 0 pred: 0.0062286262
1361: 0 pred: 0.03393163
1362: 0 pred: 0.011572931
1363: 0 pred: 0.93045115
1364: 0 pred: 0.014048546
1365: 0 pred: 0.060312055
1366: 0 pred: 0.009732749
1367: 0 pred: 0.0053330683
1368: 0 pred: 0.018025886
1369: 0 pred: 0.0055373935
1370: 0 pred: 0.007206663
1371: 0 pred: 0.007094694
1372: 0 pred: 0.009760676
1373: 0 pred: 0.005332781
1374: 0 pred: 0.006838352
1375: 0 pred: 0.025554748
1376: 0 pred: 0.016574344
1377: 0 pred: 0.015859203
1378: 0 pred: 0.015197954
1379: 0 pred: 0.006203997
1380: 0 pred: 0.11707862
1381: 0 pred: 0.26294366
1382: 0 pred: 0.010111997
1383: 0 pred: 0.015243235
1384: 0 pred: 0.017887034
1385: 0 pred: 0.048351675
1386: 0 pred: 0.005

In [25]:
proba[0,1]

-0.5386936